# MC Dropout as Bayesian Approximation Notebook

Goal: A complete workflow with prototypical functions that I'll be using for all the other workflows.

Some of the workflow will be taken from my code from Statistical NLP module.
* Ideally it's not too difficult to swap things like the dropout regime, training schedule etc
* Need to have a training manager that saves and logs data 
* Better way to define UNet - also possibly easy to switch out other networks would be nice

10th June: Complete notebook sketch 

# Installs & Imports

In [3]:
import torch
import random
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np 
import matplotlib.pyplot as plt
import os

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch import ones_like, zeros_like
# from torchsummary import summary
from tqdm.notebook import tqdm

# Detect GPU and set as default device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using {DEVICE}')

Using cpu


## Import dataset

# Process Data

## Dataset Definition

## Transformation

## DataLoader Definition

# Training Architecture

Define some key classes for training
* model
* loss metric
* accuracy metric

## UNet

## Loss

## Metric

# Training Manager

This is the class responsible handling training, evaluation, recording values during training, etc.

# Result Summary 

This class summarises training stats across many runs and has a plotting function to show the results

# Experiments Section